In [2]:
import torch
import torch_geometric
import torch_geometric.utils

a = torch.tensor([2, 2, 3]).cuda(0)
torch_geometric.utils.softmax(a)
print(a.prod())

Error: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
  File "/home/amin/miniconda3/envs/equiformer/lib/python3.8/site-packages/torch_geometric/utils/softmax.py", line 47, in softmax
        out_sum = out_sum.index_select(dim, index)
    else:
        raise NotImplementedError
        ~~~~~~~~~~~~~~~~~~~~~~~~~ <--- HERE

    return out / (out_sum + 1e-16)
RuntimeError: 


In [1]:
import time
import torch
import numpy as np
from torch_geometric.loader import DataLoader

import os
os.environ["export PYTHONNOUSERSITE"]="True"
from logger import FileLogger
from pathlib import Path
from datasets.datamodule import DataModule
from datasets.pyg.qm9 import QM9
from datasets.pyg.qm7 import QM7

#from torch_geometric.datasets import QM9
#from torch_geometric.nn import SchNet

# AMP
from contextlib import suppress
from timm.utils import NativeScaler

import nets
from nets import model_entrypoint

from timm.utils import ModelEmaV2
from timm.scheduler import create_scheduler
from optim_factory import create_optimizer

from engine import train_one_epoch, evaluate, compute_stats

# distributed training
import utils

ModelEma = ModelEmaV2
#dataset=QM7("./data",None,None,None,None,"precise3d",["u0_atom"])

In [11]:
args={}
args["dataset"]="QM7"
args["dataset_root"]="./data/qm7"
args["structure"]="precise3d"
args["dataset_args"]=["u0_atom"]
args["split"]="scaffold"
args["train_size"]=0.8
args["val_size"]=0.1
args["test_size"]=0.1
args["standardize"]=True
args["batch_size"]=16
args["inference_batch_size"]=16
args["num_workers"]=3
data = DataModule(args)
data.setup()

RDKit WARNING: [21:55:42] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [21:55:42] WARNING: not removing hydrogen atom without neighbors
[21:55:42] WARNING: not removing hydrogen atom without neighbors
[21:55:42] WARNING: not removing hydrogen atom without neighbors
computing mean and std:   0%|          | 0/342 [00:00<?, ?it/s]

train 5467, val 686, test 681


computing mean and std: 100%|██████████| 342/342 [00:04<00:00, 78.83it/s]


In [18]:
data.dataset[0].y.shape[1]

1